In [1]:
import pandas as pd
import glob
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from pathlib import Path
import cv2
from time import time
from threading import Thread
from queue import Queue

In [2]:
time_col = 'TIME'
position_cols = ['BPOGX', 'BPOGY']
reading_col = 'FPOGID'
fixation_col = 'Fixation'
fixation_id_col = 'Fixation_ID'
velocity_col = 'Velocity'
distances_col = 'Distances'
periods_col = 'Periods'
valid_col = 'BPOGV'
# new pos cols
abs_position_cols = ['ABS_X', 'ABS_Y']


In [3]:
pixel_width = 1920
pixel_height = 1080
params = [pixel_width, pixel_height]
margin = 100

green = (0, 255, 0)
red = (0, 0, 255)
yellow = (0,255,255)
blue = (255,0,0)

In [4]:
dfs = {}
for file in glob.glob("data/*/*text2*.csv"):
    df = pd.read_csv(file)
    df_cols = df.columns.values
    df_cols[4] = 'TIME'
    df.columns = df_cols
    dfs[file] = df[position_cols + [reading_col, time_col, valid_col]]

<h3>Convert relative coors to absolute

In [5]:
for filename, df in dfs.items():
    for a, r, p in zip(abs_position_cols, position_cols, params):
        df[a] = df[r]*p

<h3>Filter by BPOGV flag

In [6]:
for filename, df in dfs.items():
    df = df.loc[df[valid_col]==1,:]
    df.reset_index(drop=True, inplace=True)

<h3>Calculate distances between points and velocity

In [7]:
diff_cols = ['DIFF_'+c for c in abs_position_cols]
for filename, df in dfs.items():
    for d, c in zip(diff_cols, abs_position_cols):
        df[d] = df[c].diff().shift(-1)
    df[periods_col] = df[time_col].diff().shift(-1)
    df[distances_col] = np.square(df[diff_cols]).sum(axis=1).pow(1./2)
    df[velocity_col] = df[distances_col] / df[periods_col]
    df = df.fillna(method='ffill')


<h3>Visualize results

In [8]:
def show_last_fixation(x, df,target_col='FPOGID', estimated_col='Fixation_ID', pos_cols=['ABS_X', 'ABS_Y']):


    plt.figure(figsize=(18, 10))
    # print target groups
    plt.plot(last_groups_targets[pos_cols[0]], last_groups_targets[pos_cols[1]], 'g-')
    plt.scatter(last_groups_targets[pos_cols[0]], last_groups_targets[pos_cols[1]], marker = 'o' ,s=last_groups_targets['size']**2,c = 'g', alpha=0.4)
    
    # print estimated groups
    
    plt.plot(last_groups_est[pos_cols[0]], last_groups_est[pos_cols[1]], 'r-')
    plt.scatter(last_groups_est[pos_cols[0]], last_groups_est[pos_cols[1]], marker = 'o' ,s=last_groups_est['size']**2,c = 'r', alpha=0.4)
    plt.xlim(-margin, pixel_width + margin)
    plt.ylim(-margin, pixel_height + margin)
    plt.show()
    
    print("Targets\n",last_groups_targets)
    print("Est in target intervals\n",last_groups_est)
#     print(df[:x].groupby([estimated_col])[pos_cols].size().drop([-1], axis=0)[-5:])

In [9]:
def visualize_results(df):

    size = df.shape[0]
    sld = widgets.IntSlider(min=1,max=size,step=1,value=10)
    interact(lambda x :show_last_fixation(x ,df),  x = sld)
#     interact(show_last_fixation(x, df),  x = sld)

In [10]:
def render_result_as_video(queue,df, target_col='FPOGID', estimated_col='Fixation_ID', pos_cols=['ABS_X', 'ABS_Y'], scale = 1):
    size = df.shape[0]
#     size = 500

    for x in range(1,size):

        start = time()
        last_group = df.iloc[x][target_col]
        if last_group > 5:
            x0 = df[df[target_col]==last_group-4].first_valid_index()
        else:
            x0 = 0
        last_groups_targets = df[x0:x].groupby([target_col])[pos_cols].last()
        last_groups_targets['size'] = df[x0:x].groupby([target_col])[pos_cols].size()
    
        try:
            last_groups_est = df[x0:x].groupby([estimated_col])[pos_cols].last().drop([-1], axis=0)
            last_groups_est['size'] = df[x0:x].groupby([estimated_col])[pos_cols].size().drop([-1], axis=0)
        except KeyError:
            last_groups_est = df[x0:x].groupby([estimated_col])[pos_cols].last()
            last_groups_est['size'] = df[x0:x].groupby([estimated_col])[pos_cols].size()
        
        queue.put((last_groups_targets,last_groups_est))
        print("Computed {} frame from {}".format(x, size-1), end = '\r', flush = True)
    queue.put(None)
    print("")

In [11]:
def saver(queue,df_name, alg_name, param_name, general_folder = "result_videos", pos_cols=['ABS_X', 'ABS_Y'], scale = 1):
    result_path = Path(general_folder,alg_name,param_name)
    result_path.mkdir(parents = True, exist_ok = True)
    scaled_w = int(pixel_width/scale)
    scaled_h = int(pixel_height/scale)
    writer = cv2.VideoWriter(str(result_path / (str(df_name)+".avi")), fourcc = cv2.VideoWriter_fourcc(*'DIVX'), fps=30,
                                      frameSize=(scaled_w, scaled_h))
    white_bg = np.full((scaled_h, scaled_w, 3), 255, dtype=np.uint8)
    
    while True:
        if not queue.empty():
            res = queue.get()
            if res is not None :
                last_groups_targets, last_groups_est = res
                frame = white_bg.copy()
                # draw target groups (green)

                tar_edges = np.int32(last_groups_targets[pos_cols].to_numpy()/scale)
                cv2.polylines(frame,[tar_edges] , False, green, 2,cv2.LINE_AA)
                for point, r, gid in zip(last_groups_targets[pos_cols].to_numpy(),last_groups_targets['size'].to_numpy(),last_groups_targets.index):
                    cv2.circle(frame, tuple(np.int32(point/scale)), r, green, -1,cv2.LINE_AA)
                    cv2.putText(frame,str(gid),tuple(np.int32(point/scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.75,yellow,2,cv2.LINE_AA)
                # draw est groups (red)

                est_edges = np.int32(last_groups_est[pos_cols].to_numpy()/scale)
                cv2.polylines(frame, [est_edges], False, red, 2,cv2.LINE_AA)
                for point, r, gid in zip(last_groups_est[pos_cols].to_numpy(),last_groups_est['size'].to_numpy(),last_groups_est.index ):
                    cv2.circle(frame, tuple(np.int32(point/scale)), r, red, -1,cv2.LINE_AA)
                    cv2.putText(frame,str(gid),tuple(np.int32(point/scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.75,blue,2,cv2.LINE_AA)
                writer.write(frame)
            else:
                break
    writer.release()
    print("Video result for ",df_name, "created!")

<h1>I-DT

In [12]:
# coor [x,y]
# first point included, last - excluded
def get_dispersion_of_group(x, y, first_index, last_index):
    disp = np.abs(max(x[first_index:last_index+1]) - min(x[first_index:last_index+1]))
    disp += np.abs(max(y[first_index:last_index+1]) - min(y[first_index:last_index+1]))
    return disp

def dt_fixation_detector(duration_threshold, dispersion_threshold, dfs_2):
    for filename, df in dfs_2.items():
        size = df.shape[0]
        first_point = 0
        last_point = 1
        is_group_exists = False
        print(filename)
        print("size =", size)
        df[fixation_col] = 0
        df[fixation_id_col] = -1
        x = df[abs_position_cols[0]]
        y = df[abs_position_cols[1]]
        time = df[time_col]
        fix_id = 1
        while first_point < size and last_point < size:
#             print("FP:",first_point, "LP:",last_point)
            if is_group_exists:
                disp = get_dispersion_of_group(x, y, first_point, last_point)
                if disp <= dispersion_threshold:
                    df[fixation_col][last_point] = 1.0
                    df[fixation_id_col][last_point] = fix_id
                    last_point += 1
                else:
                    first_point = last_point
                    is_group_exists = False
                    fix_id += 1
            else:
                last_point = time[time < time[first_point]+duration_threshold].last_valid_index()+1
                disp = get_dispersion_of_group(x, y, first_point, last_point)
                if disp <= dispersion_threshold:
                    df[fixation_col][first_point:last_point+1] = 1.0
                    df[fixation_id_col][first_point:last_point+1] = fix_id
                    is_group_exists = True
                    last_point += 1
                else:
                    first_point += 1
        df[fixation_col] = df[fixation_col].fillna(0)
        print("Tracked number of groups:",df[reading_col].nunique())
        print("Estimated number of groups:",df[fixation_id_col].nunique()-1)
    return dfs_2


In [13]:
# durations = [0.1, 0.15, 0.2, 0.25, 0.3]
# disps = [100, 125, 150, 175, 200, 225]
# params = {200:[0.1, 0.2, 0.3], 300:[0.15, 0.25, 0.35], 400:[0.2, 0.3, 0.4],500:[0.2, 0.3, 0.4]}
params = {200:[0.2,0.3], 300:[0.15, 0.25, 0.35]}
for disp, durs in params.items():
    for d in durs:
        print("DISP_"+str(disp)+"_DUR_"+str(d))
        new_dfs = dt_fixation_detector(d, disp, copy.deepcopy(dfs))
        for name, df in new_dfs.items():
            queue = Queue()
            t = Thread(target = saver, args = (queue, Path(name).name.split('.')[0], "I-DT", "DISP_"+str(disp)+"_DUR_"+str(d)))
            t.start()
            render_result_as_video(queue, df)
            t.join()

DISP_200_DUR_0.2
data\Alex\Alex_text2_all_gaze_labeled.csv
size = 7337


C:\Users\Valentine\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Valentine\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Valentine\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Valentine\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A

Tracked number of groups: 254
Estimated number of groups: 268
data\Diana\Diana_text2_all_gaze_labeled.csv
size = 6015
Tracked number of groups: 235
Estimated number of groups: 275
data\Misha\Misha_text2_all_gaze_labeled.csv
size = 7224
Tracked number of groups: 238
Estimated number of groups: 256
data\Polina\Polina_text2_all_gaze_labeled.csv
size = 4722
Tracked number of groups: 189
Estimated number of groups: 192
data\Valik\Valik_text2_all_gaze_labeled.csv
size = 5760
Tracked number of groups: 195
Estimated number of groups: 191
Computed 7336 frame from 7336
Video result for  Alex_text2_all_gaze_labeled created!
Computed 6014 frame from 6014
Video result for  Diana_text2_all_gaze_labeled created!
Computed 7223 frame from 7223
Video result for  Misha_text2_all_gaze_labeled created!
Computed 4721 frame from 4721
Video result for  Polina_text2_all_gaze_labeled created!
Computed 5759 frame from 5759
Video result for  Valik_text2_all_gaze_labeled created!
DISP_200_DUR_0.3
data\Alex\Alex_te